<a href="https://colab.research.google.com/github/DinurakshanRavichandran/Visio-Glance/blob/Pre-Processed-Datasets-NLP/drusenPPFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import pandas as pd
from imblearn.over_sampling import SMOTE

# Load the dataset
file_path = '/content/drive/MyDrive/PROJECT 29/DATASETS/Synthetic_Drusen_Dataset.csv'
df = pd.read_csv(file_path)

# Handle Missing Values
# Assuming the data might have missing values, replace them with the most frequent category in each column
for column in df.columns:
    most_frequent = df[column].mode()[0]
    df[column].fillna(most_frequent, inplace=True)

# One-Hot Encode 'Smoking Status'
smoking_status_dummies = pd.get_dummies(df['Smoking Status'], prefix='Smoking')

# One-Hot Encode 'Visual Symptoms'
visual_symptoms_dummies = pd.get_dummies(df['Visual Symptoms'], prefix='Visual Symptoms')

# Combine all features
features = pd.concat([df.drop(['Diagnosis', 'Smoking Status', 'Visual Symptoms'], axis=1), smoking_status_dummies, visual_symptoms_dummies], axis=1)
target = df['Diagnosis']

# Handle Class Imbalance with SMOTE
smote = SMOTE(random_state=42)
features_smote, target_smote = smote.fit_resample(features, target)

# Reconstruct the DataFrame with resampled data
df_smote = pd.DataFrame(features_smote, columns=features.columns)
df_smote['Diagnosis'] = target_smote

# Save Preprocessed Data
preprocessed_file_path = '/content/drive/MyDrive/PROJECT 29/FINAL MODEL/Preprocessed_Drusen_Dataset.csv'
df_smote.to_csv(preprocessed_file_path, index=False)

# Display the head of the preprocessed dataset to ensure it's correct
print(df_smote.head())


   Age   BMI  Blood Pressure  Cholesterol Levels  Smoking_No  Smoking_Yes  \
0   50  25.7             158                 189       False         True   
1   53  34.5             113                 241        True        False   
2   53  26.4             153                 152        True        False   
3   89  27.7             178                 229       False         True   
4   59  25.6             113                 219        True        False   

   Visual Symptoms_Blind spots  Visual Symptoms_Blurred vision  \
0                        False                           False   
1                        False                           False   
2                        False                            True   
3                        False                           False   
4                        False                           False   

   Visual Symptoms_Distorted vision  Visual Symptoms_Light sensitivity  \
0                              True                              F

<ipython-input-6-4a115b5e9e66>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(most_frequent, inplace=True)
<ipython-input-6-4a115b5e9e66>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 